# OCR Field Extraction Pipeline

# This script processes images to extract specific fields using YOLO object detection and OCR

# Requirements:
- ultralytics
- opencv-python
- pytesseract 
- pandas
- Pillow

# Project Description:
- Utilizes YOLO (You Only Look Once) for field detection
- Implements Tesseract OCR for text extraction
- Processes document images automatically
- Exports results to structured CSV format

# Supported Field Types:
- Numeric Fields (AWB-No, Qty)
- Currency Values (FOB-Value, Total-Value)
- Text Descriptions (Goods-Description)
- Reference Numbers (Non-GST-Invoice-No)


In [20]:
%pip install ultralytics      
%pip install opencv-python    
%pip install pytesseract      
%pip install pandas           
%pip install Pillow          

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [21]:
# [Cell 1] - Imports and Logging Setup
import os
import cv2
import torch
import pandas as pd
import numpy as np
import re
from ultralytics import YOLO
import logging
from PIL import Image
import pytesseract
from datetime import datetime
from pathlib import Path

def setup_logging():
    """
    Configure logging with detailed formatting for better debugging and monitoring
    Returns:
        logger: Configured logging object
    """
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )
    return logging.getLogger('FieldExtractor')

logger = setup_logging()

In [22]:
# [Cell 2] - Text Processing Functions
def process_numeric_field(text):
    """
    Extract only numerical data from text
    Args:
        text (str): Raw text from OCR
    Returns:
        str: Extracted numerical data (only numbers, decimal points, and hyphens)
    """
    try:
        text = text.strip()
        numbers = re.findall(r'[-\d.]+', text)
        result = ''.join(numbers)
        return result
    except Exception as e:
        logger.error(f"Error processing numeric field: {str(e)}")
        return ""

def process_goods_description(text):
    """
    Process goods description text to clean and standardize format
    Args:
        text (str): Raw text from OCR
    Returns:
        str: Cleaned text as a single line with normalized spacing
    """
    try:
        text = ' '.join(text.split())
        return text.strip()
    except Exception as e:
        logger.error(f"Error processing goods description: {str(e)}")
        return ""

In [23]:
# [Cell 3] - Field Processing Configuration
FIELD_PROCESSORS = {
    'AWB-No': process_numeric_field,
    'FOB-Value-in-INR': process_numeric_field,
    'Goods-Description': process_goods_description,
    'Non-GST-Invoice-No': process_numeric_field,
    'Qty': process_numeric_field,
    'Total-Value': process_numeric_field,
    'Unit-Value': process_numeric_field
}

In [24]:
# [Cell 4] - Model Loading Function
def load_model(model_path):
    """
    Load the YOLO model and configure device (GPU/CPU)
    Args:
        model_path (str): Path to the YOLO model weights
    Returns:
        YOLO: Loaded model object
    """
    try:
        logger.info("Loading YOLO model...")
        model = YOLO(model_path)
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        model.to(device)
        logger.info(f"Model loaded successfully on {device}")
        return model
    except Exception as e:
        logger.error(f"Model loading failed: {str(e)}")
        raise


In [25]:
# [Cell 5] - Enhanced OCR Function
def extract_and_process_text(image, bbox, field_name):
    """
    Extract and process text from image region based on field type
    Args:
        image (numpy.ndarray): Input image
        bbox (tuple): Bounding box coordinates (x1, y1, x2, y2)
        field_name (str): Name of the field being processed
    Returns:
        str: Processed text according to field type
    """
    try:
        x1, y1, x2, y2 = bbox
        crop = image[y1:y2, x1:x2]
        crop_rgb = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
        pil_image = Image.fromarray(crop_rgb)
        raw_text = pytesseract.image_to_string(pil_image).strip()
        
        processor = FIELD_PROCESSORS.get(field_name)
        if processor:
            processed_text = processor(raw_text)
            logger.debug(f"Field: {field_name}, Raw: {raw_text}, Processed: {processed_text}")
            return processed_text
        return raw_text
    except Exception as e:
        logger.error(f"Text extraction failed for {field_name}: {str(e)}")
        return ""

In [26]:
# [Cell 6] - CSV Functions
def initialize_csv(output_path, field_names):
    """
    Create a new CSV file with specified headers
    Args:
        output_path (str): Path where CSV will be saved
        field_names (list): List of column headers
    Returns:
        str: Path to created CSV file
    """
    try:
        df = pd.DataFrame(columns=field_names)
        df.to_csv(output_path, index=False)
        logger.info(f"Created new CSV file at: {output_path}")
        return output_path
    except Exception as e:
        logger.error(f"Failed to create CSV file: {str(e)}")
        raise

def update_csv(csv_path, image_name, field_data):
    """
    Update CSV with processed field data
    Args:
        csv_path (str): Path to CSV file
        image_name (str): Name of processed image
        field_data (dict): Extracted field values
    """
    try:
        df = pd.read_csv(csv_path)
        new_row = pd.DataFrame([field_data], index=[image_name])
        df = pd.concat([df, new_row])
        df.to_csv(csv_path, index=False)
        logger.info(f"Updated CSV with data from {image_name}")
    except Exception as e:
        logger.error(f"Failed to update CSV: {str(e)}")
        raise

In [27]:
# [Cell 7] - Main Processing Pipeline
def process_images(input_dir, output_csv, model_path):
    """
    Main function to process images sequentially with enhanced field processing
    Args:
        input_dir (str): Directory containing input images
        output_csv (str): Path for output CSV file
        model_path (str): Path to YOLO model weights
    """
    try:
        field_names = [
            'AWB-No', 'FOB-Value-in-INR', 'Goods-Description',
            'Non-GST-Invoice-No', 'Qty', 'Total-Value', 'Unit-Value'
        ]
        
        initialize_csv(output_csv, field_names)
        model = load_model(model_path)
        
        image_files = [f for f in os.listdir(input_dir)
                      if f.lower().endswith(('.jpg', '.jpeg'))]
        total_images = len(image_files)
        
        for idx, image_file in enumerate(image_files, 1):
            logger.info(f"\nProcessing image {idx}/{total_images}: {image_file}")
            image_path = os.path.join(input_dir, image_file)
            
            image = cv2.imread(image_path)
            if image is None:
                logger.error(f"Could not read image: {image_file}")
                continue
                
            results = model(image)[0]
            field_data = {field: "" for field in field_names}
            
            for det in results.boxes.data:
                x1, y1, x2, y2, conf, cls = det
                if conf >= 0.25:  # Confidence threshold
                    field_name = field_names[int(cls)]
                    text = extract_and_process_text(
                        image,
                        (int(x1), int(y1), int(x2), int(y2)),
                        field_name
                    )
                    field_data[field_name] = text
                    logger.info(f"Processed {field_name}: {text[:50]}...")
            
            update_csv(output_csv, image_file, field_data)
            logger.info(f"Completed processing image {idx}/{total_images}")
            
        logger.info("\nProcessing completed successfully!")
        logger.info(f"Total images processed: {total_images}")
        logger.info(f"Results saved to: {output_csv}")
    except Exception as e:
        logger.error(f"Processing pipeline failed: {str(e)}")
        raise

In [28]:
# [Cell 8] - Execution
def main():
    """
    Main execution function
    Modify these paths according to your setup:
    - MODEL_PATH: Path to your trained YOLO model weights
    - INPUT_DIR: Directory containing images to process
    - OUTPUT_CSV: Directory where the results CSV should be saved
    """
    # Define paths - modify these according to your setup
    MODEL_PATH = "best.pt"
    INPUT_DIR = "Images"
    OUTPUT_CSV = os.path.join("D:\Github_Projects\VTM_Final_Src_Code_Git_Connected", f"Final_Output.csv")
    
    # Run processing pipeline
    process_images(INPUT_DIR, OUTPUT_CSV, MODEL_PATH)

In [29]:
if __name__ == "__main__":
    main()

2025-01-19 12:08:04 - INFO - Created new CSV file at: D:\Github_Projects\VTM_Final_Src_Code_Git_Connected\Final_Output.csv
2025-01-19 12:08:04 - INFO - Loading YOLO model...
2025-01-19 12:08:04 - INFO - Model loaded successfully on cpu
2025-01-19 12:08:04 - INFO - 
Processing image 1/50: 1.jpg



0: 480x640 1 AWB-No, 1 FOB-Value-in-INR, 1 Goods-Description, 1 Non-GST-Invoice-No, 1 Qty, 1 Total-Value, 1 Unit-Value, 47.0ms
Speed: 3.0ms preprocess, 47.0ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)


2025-01-19 12:08:04 - INFO - Processed Total-Value: 83.1683.16...
2025-01-19 12:08:04 - INFO - Processed Qty: 1.00...
2025-01-19 12:08:04 - INFO - Processed Non-GST-Invoice-No: -2762246-136744...
2025-01-19 12:08:04 - INFO - Processed FOB-Value-in-INR: 6835.75...
2025-01-19 12:08:04 - INFO - Processed Unit-Value: 83.1683.16...
2025-01-19 12:08:05 - INFO - Processed Goods-Description: Goods Description EuropeanLinenWaffleDuvetCover Se...
2025-01-19 12:08:05 - INFO - Processed AWB-No: 3292697236...
2025-01-19 12:08:05 - INFO - Updated CSV with data from 1.jpg
2025-01-19 12:08:05 - INFO - Completed processing image 1/50
2025-01-19 12:08:05 - INFO - 
Processing image 2/50: 10.jpg



0: 480x640 1 AWB-No, 1 FOB-Value-in-INR, 1 Goods-Description, 1 Non-GST-Invoice-No, 1 Qty, 1 Total-Value, 1 Unit-Value, 47.0ms
Speed: 2.0ms preprocess, 47.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)


2025-01-19 12:08:05 - INFO - Processed Total-Value: 83.1683.16...
2025-01-19 12:08:05 - INFO - Processed Qty: 1.00...
2025-01-19 12:08:05 - INFO - Processed Non-GST-Invoice-No: -2775735-138220...
2025-01-19 12:08:05 - INFO - Processed FOB-Value-in-INR: 6835.75...
2025-01-19 12:08:05 - INFO - Processed Unit-Value: 83.1683.16...
2025-01-19 12:08:06 - INFO - Processed Goods-Description: Goods Description EuropeanLinenWaffleDuvetCover Se...
2025-01-19 12:08:06 - INFO - Processed AWB-No: 1048189015...
2025-01-19 12:08:06 - INFO - Updated CSV with data from 10.jpg
2025-01-19 12:08:06 - INFO - Completed processing image 2/50
2025-01-19 12:08:06 - INFO - 
Processing image 3/50: 11.jpg


...

2025-01-19 12:08:51 - INFO - Processed Total-Value: 88.6088.60...
2025-01-19 12:08:51 - INFO - Processed Qty: 1.00...
2025-01-19 12:08:51 - INFO - Processed Non-GST-Invoice-No: -2768068-137367...
2025-01-19 12:08:51 - INFO - Processed Unit-Value: 88.6088.60...
2025-01-19 12:08:51 - INFO - Processed FOB-Value-in-INR: 7282.92...
2025-01-19 12:08:51 - INFO - Processed Goods-Description: Goods Description EuropeanLinenWaffleDuvetCover Ki...
2025-01-19 12:08:52 - INFO - Processed AWB-No: 1796820421...
2025-01-19 12:08:52 - INFO - Updated CSV with data from 7.jpg
2025-01-19 12:08:52 - INFO - Completed processing image 48/50
2025-01-19 12:08:52 - INFO - 
Processing image 49/50: 8.jpg



0: 480x640 1 AWB-No, 1 FOB-Value-in-INR, 1 Goods-Description, 1 Non-GST-Invoice-No, 1 Qty, 1 Total-Value, 1 Unit-Value, 44.0ms
Speed: 3.0ms preprocess, 44.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)


2025-01-19 12:08:52 - INFO - Processed Total-Value: 83.1683.16...
2025-01-19 12:08:52 - INFO - Processed Qty: 1.00...
2025-01-19 12:08:52 - INFO - Processed Non-GST-Invoice-No: -2774519-138157...
2025-01-19 12:08:52 - INFO - Processed FOB-Value-in-INR: 6835.75...
2025-01-19 12:08:52 - INFO - Processed Unit-Value: 83.1683.16...
2025-01-19 12:08:52 - INFO - Processed Goods-Description: Goods Description EuropeanLinenWaffleDuvetCover Se...
2025-01-19 12:08:52 - INFO - Processed AWB-No: 1202410650...
2025-01-19 12:08:52 - INFO - Updated CSV with data from 8.jpg
2025-01-19 12:08:52 - INFO - Completed processing image 49/50
2025-01-19 12:08:52 - INFO - 
Processing image 50/50: 9.jpg



0: 480x640 1 AWB-No, 1 FOB-Value-in-INR, 1 Goods-Description, 1 Non-GST-Invoice-No, 1 Qty, 1 Total-Value, 1 Unit-Value, 45.0ms
Speed: 2.0ms preprocess, 45.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)


2025-01-19 12:08:53 - INFO - Processed Total-Value: 69.9169.91...
2025-01-19 12:08:53 - INFO - Processed Qty: 1.00...
2025-01-19 12:08:53 - INFO - Processed FOB-Value-in-INR: 5746.6...
2025-01-19 12:08:53 - INFO - Processed Non-GST-Invoice-No: -2774604-138147...
2025-01-19 12:08:53 - INFO - Processed Unit-Value: 69.9169.91...
2025-01-19 12:08:53 - INFO - Processed Goods-Description: Goods Description EuropeanLinenWaffleDuvetCover Fu...
2025-01-19 12:08:53 - INFO - Processed AWB-No: 1202401605...
2025-01-19 12:08:53 - INFO - Updated CSV with data from 9.jpg
2025-01-19 12:08:53 - INFO - Completed processing image 50/50
2025-01-19 12:08:53 - INFO - 
Processing completed successfully!
2025-01-19 12:08:53 - INFO - Total images processed: 50
2025-01-19 12:08:53 - INFO - Results saved to: D:\Github_Projects\VTM_Final_Src_Code_Git_Connected\Final_Output.csv
